<a href="https://colab.research.google.com/github/VictorAlexandr/Data-Science-Projects/blob/main/Sistema_de_Recomenda%C3%A7%C3%A3o_de_Produtos_para_E_commerce.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Título:** Sistema de Recomendação de Produtos para E-commerce

**Objetivo:** Desenvolver um sistema de recomendação de produtos que sugira produtos para os usuários com base em suas preferências e compras anteriores.

**Dataset:** Online Retail II (https://www.kaggle.com/carrie1/online-retail-ii)

**Tarefa:**

- Desenvolver um sistema de recomendação de produtos que sugira produtos para os usuários com base em suas preferências e compras anteriores.

- Utilizar a métrica de similaridade coseno para calcular a similaridade entre os usuários.

- Recomendar produtos para os usuários com base na similaridade calculada.

**Benefícios:**

- Melhorar a experiência do usuário no e-commerce.
- Aumentar as vendas e a fidelidade dos clientes.
- Oferecer produtos personalizados para os usuários.

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
# Carregar o dataset
df = pd.read_excel('online_retail_II.xlsx')

In [3]:
# Remover linhas com valores faltantes
df.dropna(inplace=True)

In [4]:
# Converter a coluna 'InvoiceDate' para datetime
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])

In [7]:
# Renomear a coluna 'Customer ID' para 'CustomerID'
df.rename(columns={'Customer ID': 'CustomerID'}, inplace=True)

In [8]:
# Criar uma coluna 'UserId' única para cada cliente
df['UserId'] = df['CustomerID']

In [9]:
# Criar uma coluna 'ProductId' única para cada produto
df['ProductId'] = df['StockCode']

In [10]:
# Criar uma coluna 'Quantity' com a quantidade comprada
df['Quantity'] = df['Quantity']

In [12]:
# Criar uma coluna 'Preço' com o preço do produto
df['Preço'] = df['Price']

In [15]:
# Remover colunas desnecessárias
df.drop(['CustomerID', 'StockCode', 'Price', 'Invoice'], axis=1, inplace=True)

In [24]:
# Criar uma matriz de utilização com os usuários como linhas e os produtos como colunas
user_item_matrix = pd.pivot_table(df, index='UserId', columns='ProductId', values='Quantity', aggfunc='sum', fill_value=0)

In [25]:
# Calcular a similaridade entre os usuários utilizando a métrica de similaridade coseno
similarity_matrix = cosine_similarity(user_item_matrix)

def recommend_products(user_id, num_recs=5):
    user_similarities = similarity_matrix[user_id]
    similar_user_indices = np.argsort(-user_similarities)[:num_recs]
    recommended_products = []
    for idx in similar_user_indices:
        products = user_item_matrix.columns[user_item_matrix.iloc[idx] > 0]
        recommended_products.extend(products)
    recommended_products = [prod for prod in recommended_products if prod not in user_item_matrix.columns[user_item_matrix.iloc[user_id] > 0]]
    return recommended_products[:num_recs]

In [27]:
# Testar o algoritmo de recomendação para um usuário
num_usuarios = similarity_matrix.shape[0]
user_id = np.random.randint(0, num_usuarios)
recommended_products = recommend_products(user_id)
print(recommended_products)

[22079, 22080, 35979, 84946, 85066]


O resultado **[22079, 22080, 35979, 84946, 85066]** representa os IDs dos produtos recomendados para o usuário gerado aleatoriamente. Esses produtos são os mais semelhantes aos que o usuário já comprou ou mostrou interesse, com base na análise de similaridade entre os usuários.

Agora, pode usar esses IDs para obter mais informações sobre os produtos recomendados, como o nome, descrição, preço, etc. Basta acessar o DataFrame df e filtrar os produtos com esses IDs. Por exemplo:

In [28]:
produtos_recomendados = df[df['ProductId'].isin([22079, 22080, 35979, 84946, 85066])]
print(produtos_recomendados)

                            Description  Quantity         InvoiceDate  \
172         CREAM SWEETHEART MINI CHEST         2 2009-12-01 10:06:00   
184     ANTIQUE SILVER TEA GLASS ETCHED       -12 2009-12-01 10:33:00   
193     ANTIQUE SILVER TEA GLASS ETCHED        12 2009-12-01 10:36:00   
412     ANTIQUE SILVER TEA GLASS ETCHED        12 2009-12-01 11:37:00   
450           RIBBON REEL SPOTS DESIGN         10 2009-12-01 11:41:00   
...                                 ...       ...                 ...   
523616      CREAM SWEETHEART MINI CHEST         1 2010-12-09 14:08:00   
524361      CREAM SWEETHEART MINI CHEST         2 2010-12-09 14:25:00   
525332       RIBBON REEL HEARTS DESIGN          5 2010-12-09 19:23:00   
525344  ANTIQUE SILVER TEA GLASS ETCHED         6 2010-12-09 19:23:00   
525369  ANTIQUE SILVER TEA GLASS ETCHED         2 2010-12-09 19:28:00   

               Country   UserId ProductId  Preço  
172     United Kingdom  13758.0     85066  12.75  
184          Australi

Para melhorar a recomendação, pode considerar adicionar mais critérios de filtro, como:

- Excluir produtos que o usuário já comprou anteriormente
- Excluir produtos que não sejam relevantes para o usuário atual
- Incluir produtos que sejam mais populares ou mais bem avaliados


Além disso, pode considerar usar técnicas de recomendação mais avançadas, como:

- Collaborative Filtering (CF)
- Content-Based Filtering (CBF)
- Hybrid Recommender Systems